In [7]:
-- CREATE DIMENSION_DEVISE
IF OBJECT_ID('schema_gold.DIMENSION_DEVISE', 'V') IS NOT NULL 
    DROP VIEW [schema_gold].[DIMENSION_DEVISE]

CREATE VIEW schema_gold.DIMENSION_DEVISE AS
SELECT 
    ROW_NUMBER() OVER (ORDER BY devise) AS id_devise,
    devise
FROM (
    SELECT DISTINCT devise
    FROM schema_silver.silver_core_bank
) AS DIMENSION_DEVISE;

-- CREATE DIMENSION_PAYS
IF OBJECT_ID('schema_gold.DIMENSION_PAYS', 'V') IS NOT NULL 
    DROP VIEW [schema_gold].[DIMENSION_PAYS]

CREATE VIEW schema_gold.DIMENSION_PAYS AS
SELECT 
    ROW_NUMBER() OVER (ORDER BY code_pays) AS id_pays,
    code_pays
FROM (
    SELECT DISTINCT code_pays
    FROM schema_silver.silver_core_bank
) AS DIMENSION_PAYS;

-- CREATE DIMENSION TERMINAL
IF OBJECT_ID('schema_gold.DIMENSION_TERMINAL', 'V') IS NOT NULL 
    DROP VIEW [schema_gold].[DIMENSION_TERMINAL]


CREATE VIEW schema_gold.DIMENSION_TERMINAL AS
SELECT 
    ROW_NUMBER() OVER (ORDER BY device_of_trx) AS id_terminal,
    device_of_trx as terminal
FROM (
    SELECT DISTINCT device_of_trx
    FROM schema_silver.silver_core_bank
) AS DIMENSION_TERMINAL;


-- CREATE DIMENSION_AGENCE
IF OBJECT_ID('schema_gold.DIMENSION_AGENCE', 'V') IS NOT NULL
    DROP VIEW schema_gold.DIMENSION_AGENCE;

CREATE VIEW schema_gold.DIMENSION_AGENCE AS
SELECT DISTINCT agence_id, agence_nom
FROM schema_silver.silver_core_bank;


-- CREATE DIMENSION_TYPE_TRX
IF OBJECT_ID('schema_gold.DIMENSION_TYPE_TRX', 'V') IS NOT NULL
    DROP VIEW schema_gold.DIMENSION_TYPE_TRX;

CREATE VIEW schema_gold.DIMENSION_TYPE_TRX AS
SELECT 
    ROW_NUMBER() OVER (ORDER BY type_trx) AS id_type_trx, 
    type_trx
FROM (
    SELECT DISTINCT type_trx
    FROM schema_silver.silver_core_bank
) AS t

--CREATE DIMENSION_DATE
IF OBJECT_ID('schema_gold.DIMENSION_DATE', 'U') IS NOT NULL
    DROP VIEW schema_gold.DIMENSION_DATE;

CREATE VIEW schema_gold.DIMENSION_DATE AS
SELECT 
    ROW_NUMBER() OVER (ORDER BY date_trx) as id_date,
    date_trx,
    MONTH(date_trx) AS mois, 
    DAY(date_trx) AS jours, 
    YEAR(date_trx) as annee
FROM (
    SELECT 
        DISTINCT CONVERT(DATE, date_trx, 120) as date_trx
    FROM schema_silver.silver_core_bank

) AS t


-- CREATE DIMENSION_CLIENT
IF OBJECT_ID('schema_gold.DIMENSION_CLIENT', 'V') IS NOT NULL
    DROP VIEW schema_gold.DIMENSION_CLIENT

CREATE  VIEW schema_gold.DIMENSION_CLIENT AS
SELECT *
FROM schema_silver.silver_crm;


-- CREATE FACTTABLE
IF OBJECT_ID('schema_gold.FACTTABLE', 'V') IS NOT NULL
    DROP VIEW schema_gold.FACTTABLE;


CREATE VIEW schema_gold.FACTTABLE AS 
SELECT 
    CONVERT(DATE, t.date_trx, 120) as date, 
    t.num_compt_init,
    t.num_compt_dest,
    t.montant_trx,
    dim_agence.agence_id, 
    dim_devise.id_devise, 
    dim_type_trx.id_type_trx,
    dim_terminal.id_terminal,
    dim_pays.id_pays
FROM schema_silver.silver_core_bank as t
LEFT JOIN (SELECT * FROM schema_gold.DIMENSION_AGENCE) as dim_agence
ON t.agence_id = dim_agence.agence_id
LEFT JOIN (SELECT * FROM schema_gold.DIMENSION_DEVISE) as dim_devise
ON t.devise =  dim_devise.devise
LEFT JOIN (SELECT * FROM schema_gold.DIMENSION_TYPE_TRX) as dim_type_trx
ON t.type_trx = dim_type_trx.type_trx
LEFT JOIN (SELECT * FROM schema_gold.DIMENSION_TERMINAL) as dim_terminal
ON t.device_of_trx = dim_terminal.terminal
LEFT JOIN (select * from schema_gold.DIMENSION_PAYS) as dim_pays
ON t.code_pays =  dim_pays.code_pays

DatawarehouseExecuteSqlError: The specified schema name "schema_gold" either does not exist or you do not have permission to use it. Data warehouse trace id: 1a6bf101-0ea6-47c1-b9b6-c322fea758d4